# Data Load

In [1]:
import pandas as pd

In [2]:
from modules.data_store import data_store

There is data stored
Last data fetch completed


In [3]:
data_store = data_store()

Load the saved pickle file


In [4]:
trial = data_store.return_last_trial()
df = data_store.return_dict_df()[trial]

## target 값 - input변수 norm변환후 계산

In [5]:
import modules.config as config
input_cols = config.trial_4_input_cols
input_cols

['DAT',
 'obs_hour',
 'norm_change_내부온도관측치',
 'norm_내부습도관측치',
 'norm_co2관측치',
 'norm_ec관측치',
 'norm_시간당분무량',
 'norm_일간누적분무량',
 'norm_change_시간당백색광량',
 'norm_change_일간누적백색광량',
 'norm_change_시간당적색광량',
 'norm_change_일간누적적색광량',
 'norm_change_시간당청색광량',
 'norm_change_일간누적청색광량',
 'norm_change_시간당총광량',
 'norm_change_일간누적총광량']

## EBM

In [6]:
base_df = df.copy()

In [7]:
train_df = base_df[base_df['train/test']=='CASE']
test_df = base_df[base_df['train/test']=='TEST']

In [8]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train_df[input_cols]
y_train = train_df['target']

ebm_1 = ExplainableBoostingRegressor()
ebm_1.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_hour',
                                            'norm_change_내부온도관측치',
                                            'norm_내부습도관측치', 'norm_co2관측치',
                                            'norm_ec관측치', 'norm_시간당분무량',
                                            'norm_일간누적분무량',
                                            'norm_change_시간당백색광량',
                                            'norm_change_일간누적백색광량',
                                            'norm_change_시간당적색광량',
                                            'norm_change_일간누적적색광량',
                                            'norm_change_시간당청색광량',
                                            'norm_change_일간누적청색광량',
                                            'norm_change_시간당총광량',
                                            'norm_change_일간누적총광량',
                                            'DAT x norm_change_시...
                             feature_types=['continuous'

In [9]:
x_predict = ebm_1.predict(X_train)
X_train['predict_d1'] = pd.DataFrame(x_predict)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
X_train['predict_d1'].head(24).sum()

-0.7747895654864869

In [11]:
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

X_train['predict_d1'] = X_train['predict_d1'].apply(predict_rslt_weight)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
X_train['predict_d1'].head(24).sum()

1.1063112633191683

In [13]:
ebm_2 = ExplainableBoostingRegressor()
ebm_2.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_hour',
                                            'norm_change_내부온도관측치',
                                            'norm_내부습도관측치', 'norm_co2관측치',
                                            'norm_ec관측치', 'norm_시간당분무량',
                                            'norm_일간누적분무량',
                                            'norm_change_시간당백색광량',
                                            'norm_change_일간누적백색광량',
                                            'norm_change_시간당적색광량',
                                            'norm_change_일간누적적색광량',
                                            'norm_change_시간당청색광량',
                                            'norm_change_일간누적청색광량',
                                            'norm_change_시간당총광량',
                                            'norm_change_일간누적총광량', 'predict_d1',
                                            'norm...
                             feature_types=['continuous',

In [15]:
x_predict = ebm_2.predict(X_train)
pd.DataFrame(x_predict)[:24].sum()

0    1.629731
dtype: float64

# test predict

In [16]:
X_test = test_df[input_cols]
y_test = test_df['target']


X_test['predict_d1'] = ebm_1.predict(X_test)
X_test['predict_d1'] = X_test['predict_d1'].apply(predict_rslt_weight)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
test_df['predicted_weight_g'] = ebm_2.predict(X_test)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
base_rslt_df = test_df[['DAT','file_num','predicted_weight_g']].groupby(['DAT','file_num']).sum().reset_index()
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

base_rslt_df['predicted_weight_g'] = base_rslt_df['predicted_weight_g'].apply(predict_rslt_weight)
# base_rslt_df = base_rslt_df.set_index('DAT')
base_rslt_df

,DAT,file_num,predicted_weight_g
0,1,01,2.342505
1,1,02,1.205953
2,1,03,0.000000
3,1,04,0.971466
4,1,05,1.416712
...,...,...,...
135,28,01,96.821102
136,28,02,87.484524
137,28,03,38.462209
138,28,04,116.400319


In [20]:
file_num_list = set(base_rslt_df['file_num'])
test_target_list = [] 
file_path = './open/sample_submission/'
for file_num in file_num_list:
    test_target_file = file_path+'TEST_'+file_num+'.csv'
    print(test_target_file)
    test_target_list.append(test_target_file)
    base_rslt_df[base_rslt_df['file_num']==file_num][['DAT','predicted_weight_g']].to_csv(test_target_file,index=False)
    

./open/sample_submission/TEST_05.csv
./open/sample_submission/TEST_03.csv
./open/sample_submission/TEST_02.csv
./open/sample_submission/TEST_01.csv
./open/sample_submission/TEST_04.csv


- 성능이 8.5로 하락

---